In [1]:
import os, numpy as np
import pandas as pd



from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

/home/antoine/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Build a Keras Model

In [2]:
# shameless copy of keras example : https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py

sample = True

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


if(sample):
    indices = np.random.choice(x_train.shape[0], x_train.shape[0] // 10, replace=False)
    x_train = x_train[indices, : , :, :]
    y_train = y_train[indices]
    indices = np.random.choice(x_test.shape[0], x_test.shape[0] // 10, replace=False)
    x_test = x_test[indices, : , :, :]
    y_test = y_test[indices]


print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')



# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



x_train shape: (6000, 28, 28, 1)
6000 train samples
1000 test samples


In [12]:

def create_model():
    model = Sequential()
    model.add(Conv2D(8, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    # model.add(Conv2D(4, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])

    return model


In [13]:

from keras.wrappers.scikit_learn import KerasClassifier

clf = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=1)

clf.fit(x_train, y_train ,
        batch_size=batch_size,
        epochs=12,
        verbose=1,
        validation_data=(x_test, y_test))


Train on 6000 samples, validate on 1000 samples
Epoch 1/12
6000/6000 [==============================] - 4s 591us/step - loss: 1.7582 - acc: 0.4460 - val_loss: 0.8198 - val_acc: 0.8450
Epoch 2/12
6000/6000 [==============================] - 3s 562us/step - loss: 0.8134 - acc: 0.7495 - val_loss: 0.4193 - val_acc: 0.9050
Epoch 3/12
6000/6000 [==============================] - 3s 573us/step - loss: 0.5875 - acc: 0.8193 - val_loss: 0.3247 - val_acc: 0.9220
Epoch 4/12
6000/6000 [==============================] - 3s 577us/step - loss: 0.4855 - acc: 0.8533 - val_loss: 0.2750 - val_acc: 0.9290
Epoch 5/12
6000/6000 [==============================] - 3s 569us/step - loss: 0.4353 - acc: 0.8687 - val_loss: 0.2453 - val_acc: 0.9370
Epoch 6/12
6000/6000 [==============================] - 3s 566us/step - loss: 0.3885 - acc: 0.8817 - val_loss: 0.2308 - val_acc: 0.9400
Epoch 7/12
6000/6000 [==============================] - 3s 569us/step - loss: 0.3557 - acc: 0.8908 - val_loss: 0.2074 - val_acc: 0.9430


In [15]:
print(x_test.shape)
preds = clf.predict(x_test[0,:].reshape(1, 28 , 28, 1))
print(preds)


(1000, 28, 28, 1)
1/1 [==============================] - 0s 2ms/step
[2]


# Generate SQL Code from the Model

In [16]:
import json, requests, base64, dill as pickle, sys



sys.setrecursionlimit(200000)
pickle.settings['recurse'] = False

# no luck for the web service... pickling feature of tensorflow and/or keras objects seems not to be a priority.
# there is a lot of github issues in the two projects when I search for pickle keyword!!!.

def test_ws_sql_gen(pickle_data):
    WS_URL="http://localhost:1888/model"
    b64_data = base64.b64encode(pickle_data).decode('utf-8')
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"postgresql"}
    r = requests.post(WS_URL, json=data)
    print(r.__dict__)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;



def test_sql_gen(keras_regressor , metadata):
    import sklearn2sql.PyCodeGenerator as codegen
    cg1 = codegen.cAbstractCodeGenerator();
    lSQL = cg1.generateCodeWithMetadata(clf, metadata, dsn = None, dialect = "postgresql");
    return lSQL[0]


In [17]:
# commented .. see above
# pickle_data = pickle.dumps(clf)
# lSQL = test_ws_sql_gen(pickle_data)
# print(lSQL[0:2000])

In [18]:
lMetaData = {}
NC = x_test.shape[1] *  x_test.shape[2] *  x_test.shape[3]
lMetaData['features'] = ["X_" + str(x+1)  for x in range(0 , NC)]

lMetaData["targets"] = ['TGT']
lMetaData['primary_key'] = 'KEY'
lMetaData['table'] = 'mnist'

    
lSQL = test_sql_gen(clf , lMetaData)


cGenerationWrapperFactory::createWrapper() <class 'keras.wrappers.scikit_learn.KerasClassifier'>
cClassifierMixin_CodeGenWrapper::setObject <class 'keras.wrappers.scikit_learn.KerasClassifier'>
BACKEND_DIALECT postgresql
CREATING_DATABASE_BACKEND_DSN_DIALECT 1.2.2 None postgresql
KERAS_GENERATE_EXPRESSION_START <class 'sqlalchemy.sql.selectable.Alias'> <class 'sklearn2sql.GenericModels.Keras.cSerializedKeras'>
{'mKerasData': <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f77349c86a0>, 'mFeatureNames': ['X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 'X_7', 'X_8', 'X_9', 'X_10', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17', 'X_18', 'X_19', 'X_20', 'X_21', 'X_22', 'X_23', 'X_24', 'X_25', 'X_26', 'X_27', 'X_28', 'X_29', 'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35', 'X_36', 'X_37', 'X_38', 'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44', 'X_45', 'X_46', 'X_47', 'X_48', 'X_49', 'X_50', 'X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56', 'X_57', 'X_58', 'X_59', 'X_60', 'X_61', 'X

ABSTRACT_GENERATE_CTE layer_conv2d_2_Filter0 layer_conv2d_2_Filter0 677
1
[[ 0.0999394   0.16322748  0.30838144]
 [-0.04505034  0.33969465  0.14073877]
 [-0.1790891   0.04001456  0.16900565]]
-0.011353365
ABSTRACT_GENERATE_CTE layer_conv2d_2_Filter1 layer_conv2d_2_Filter1 677
2
[[-0.21952663 -0.14317936  0.3043259 ]
 [-0.04308624  0.30865562  0.26139638]
 [ 0.27910182  0.03403345  0.07984909]]
-0.009909085
ABSTRACT_GENERATE_CTE layer_conv2d_2_Filter2 layer_conv2d_2_Filter2 677
3
[[ 0.03582093 -0.0797584   0.08646487]
 [ 0.28229252  0.3059959  -0.20103467]
 [ 0.1825967   0.20045725  0.2316563 ]]
-0.012215332
ABSTRACT_GENERATE_CTE layer_conv2d_2_Filter3 layer_conv2d_2_Filter3 677
4
[[-0.1567965  -0.5117769  -0.11086897]
 [-0.59734243 -0.06824617  0.5446455 ]
 [-0.37175873  0.02451065  0.3519969 ]]
0.079545185
ABSTRACT_GENERATE_CTE layer_conv2d_2_Filter4 layer_conv2d_2_Filter4 677
5
[[-0.38759363 -0.26824474 -0.16097717]
 [ 0.14778689  0.23413867  0.2946185 ]
 [ 0.15048231  0.12601744 -0.

FLATTEN_JOIN_COL 2 3 7 1212 flatten_join.output_7_2_3
FLATTEN_JOIN_COL 2 4 0 30 flatten_join.output_0_2_4
FLATTEN_JOIN_COL 2 4 1 199 flatten_join.output_1_2_4
FLATTEN_JOIN_COL 2 4 2 368 flatten_join.output_2_2_4
FLATTEN_JOIN_COL 2 4 3 537 flatten_join.output_3_2_4
FLATTEN_JOIN_COL 2 4 4 706 flatten_join.output_4_2_4
FLATTEN_JOIN_COL 2 4 5 875 flatten_join.output_5_2_4
FLATTEN_JOIN_COL 2 4 6 1044 flatten_join.output_6_2_4
FLATTEN_JOIN_COL 2 4 7 1213 flatten_join.output_7_2_4
FLATTEN_JOIN_COL 2 5 0 31 flatten_join.output_0_2_5
FLATTEN_JOIN_COL 2 5 1 200 flatten_join.output_1_2_5
FLATTEN_JOIN_COL 2 5 2 369 flatten_join.output_2_2_5
FLATTEN_JOIN_COL 2 5 3 538 flatten_join.output_3_2_5
FLATTEN_JOIN_COL 2 5 4 707 flatten_join.output_4_2_5
FLATTEN_JOIN_COL 2 5 5 876 flatten_join.output_5_2_5
FLATTEN_JOIN_COL 2 5 6 1045 flatten_join.output_6_2_5
FLATTEN_JOIN_COL 2 5 7 1214 flatten_join.output_7_2_5
FLATTEN_JOIN_COL 2 6 0 32 flatten_join.output_0_2_6
FLATTEN_JOIN_COL 2 6 1 201 flatten_join.outp

FLATTEN_JOIN_COL 3 11 5 895 flatten_join.output_5_3_11
FLATTEN_JOIN_COL 3 11 6 1064 flatten_join.output_6_3_11
FLATTEN_JOIN_COL 3 11 7 1233 flatten_join.output_7_3_11
FLATTEN_JOIN_COL 3 12 0 51 flatten_join.output_0_3_12
FLATTEN_JOIN_COL 3 12 1 220 flatten_join.output_1_3_12
FLATTEN_JOIN_COL 3 12 2 389 flatten_join.output_2_3_12
FLATTEN_JOIN_COL 3 12 3 558 flatten_join.output_3_3_12
FLATTEN_JOIN_COL 3 12 4 727 flatten_join.output_4_3_12
FLATTEN_JOIN_COL 3 12 5 896 flatten_join.output_5_3_12
FLATTEN_JOIN_COL 3 12 6 1065 flatten_join.output_6_3_12
FLATTEN_JOIN_COL 3 12 7 1234 flatten_join.output_7_3_12
FLATTEN_JOIN_COL 4 0 0 52 flatten_join.output_0_4_0
FLATTEN_JOIN_COL 4 0 1 221 flatten_join.output_1_4_0
FLATTEN_JOIN_COL 4 0 2 390 flatten_join.output_2_4_0
FLATTEN_JOIN_COL 4 0 3 559 flatten_join.output_3_4_0
FLATTEN_JOIN_COL 4 0 4 728 flatten_join.output_4_4_0
FLATTEN_JOIN_COL 4 0 5 897 flatten_join.output_5_4_0
FLATTEN_JOIN_COL 4 0 6 1066 flatten_join.output_6_4_0
FLATTEN_JOIN_COL 4 0 

FLATTEN_JOIN_COL 7 10 5 946 flatten_join.output_5_7_10
FLATTEN_JOIN_COL 7 10 6 1115 flatten_join.output_6_7_10
FLATTEN_JOIN_COL 7 10 7 1284 flatten_join.output_7_7_10
FLATTEN_JOIN_COL 7 11 0 102 flatten_join.output_0_7_11
FLATTEN_JOIN_COL 7 11 1 271 flatten_join.output_1_7_11
FLATTEN_JOIN_COL 7 11 2 440 flatten_join.output_2_7_11
FLATTEN_JOIN_COL 7 11 3 609 flatten_join.output_3_7_11
FLATTEN_JOIN_COL 7 11 4 778 flatten_join.output_4_7_11
FLATTEN_JOIN_COL 7 11 5 947 flatten_join.output_5_7_11
FLATTEN_JOIN_COL 7 11 6 1116 flatten_join.output_6_7_11
FLATTEN_JOIN_COL 7 11 7 1285 flatten_join.output_7_7_11
FLATTEN_JOIN_COL 7 12 0 103 flatten_join.output_0_7_12
FLATTEN_JOIN_COL 7 12 1 272 flatten_join.output_1_7_12
FLATTEN_JOIN_COL 7 12 2 441 flatten_join.output_2_7_12
FLATTEN_JOIN_COL 7 12 3 610 flatten_join.output_3_7_12
FLATTEN_JOIN_COL 7 12 4 779 flatten_join.output_4_7_12
FLATTEN_JOIN_COL 7 12 5 948 flatten_join.output_5_7_12
FLATTEN_JOIN_COL 7 12 6 1117 flatten_join.output_6_7_12
FLATT

FLATTEN_JOIN_COL 11 0 2 481 flatten_join.output_2_11_0
FLATTEN_JOIN_COL 11 0 3 650 flatten_join.output_3_11_0
FLATTEN_JOIN_COL 11 0 4 819 flatten_join.output_4_11_0
FLATTEN_JOIN_COL 11 0 5 988 flatten_join.output_5_11_0
FLATTEN_JOIN_COL 11 0 6 1157 flatten_join.output_6_11_0
FLATTEN_JOIN_COL 11 0 7 1326 flatten_join.output_7_11_0
FLATTEN_JOIN_COL 11 1 0 144 flatten_join.output_0_11_1
FLATTEN_JOIN_COL 11 1 1 313 flatten_join.output_1_11_1
FLATTEN_JOIN_COL 11 1 2 482 flatten_join.output_2_11_1
FLATTEN_JOIN_COL 11 1 3 651 flatten_join.output_3_11_1
FLATTEN_JOIN_COL 11 1 4 820 flatten_join.output_4_11_1
FLATTEN_JOIN_COL 11 1 5 989 flatten_join.output_5_11_1
FLATTEN_JOIN_COL 11 1 6 1158 flatten_join.output_6_11_1
FLATTEN_JOIN_COL 11 1 7 1327 flatten_join.output_7_11_1
FLATTEN_JOIN_COL 11 2 0 145 flatten_join.output_0_11_2
FLATTEN_JOIN_COL 11 2 1 314 flatten_join.output_1_11_2
FLATTEN_JOIN_COL 11 2 2 483 flatten_join.output_2_11_2
FLATTEN_JOIN_COL 11 2 3 652 flatten_join.output_3_11_2
FLATTE

ABSTRACT_GENERATE_CTE score_soft_max_step1 score_soft_max_step1 21
ABSTRACT_GENERATE_CTE score_class_union_soft score_class_union_soft 3
ABSTRACT_GENERATE_CTE score_soft_max score_soft_max 1
['score_soft_max.KEY_7_7', 'score_soft_max."Score_0"', 'score_soft_max."exp_Score_0"', 'score_soft_max."Score_1"', 'score_soft_max."exp_Score_1"', 'score_soft_max."Score_2"', 'score_soft_max."exp_Score_2"', 'score_soft_max."Score_3"', 'score_soft_max."exp_Score_3"', 'score_soft_max."Score_4"', 'score_soft_max."exp_Score_4"', 'score_soft_max."Score_5"', 'score_soft_max."exp_Score_5"', 'score_soft_max."Score_6"', 'score_soft_max."exp_Score_6"', 'score_soft_max."Score_7"', 'score_soft_max."exp_Score_7"', 'score_soft_max."Score_8"', 'score_soft_max."exp_Score_8"', 'score_soft_max."Score_9"', 'score_soft_max."exp_Score_9"', 'score_soft_max.KEY_7_7_sum', 'score_soft_max."sum_ExpScore"']
ABSTRACT_GENERATE_CTE layer_softmax layer_softmax 11
ABSTRACT_GENERATE_CTE orig_cte orig_cte 34
ABSTRACT_GENERATE_CTE s

In [19]:
print(lSQL[0:50000])

WITH keras_input AS 
(SELECT "ADS"."KEY" AS "KEY", "ADS"."X_1" AS "X_1", "ADS"."X_2" AS "X_2", "ADS"."X_3" AS "X_3", "ADS"."X_4" AS "X_4", "ADS"."X_5" AS "X_5", "ADS"."X_6" AS "X_6", "ADS"."X_7" AS "X_7", "ADS"."X_8" AS "X_8", "ADS"."X_9" AS "X_9", "ADS"."X_10" AS "X_10", "ADS"."X_11" AS "X_11", "ADS"."X_12" AS "X_12", "ADS"."X_13" AS "X_13", "ADS"."X_14" AS "X_14", "ADS"."X_15" AS "X_15", "ADS"."X_16" AS "X_16", "ADS"."X_17" AS "X_17", "ADS"."X_18" AS "X_18", "ADS"."X_19" AS "X_19", "ADS"."X_20" AS "X_20", "ADS"."X_21" AS "X_21", "ADS"."X_22" AS "X_22", "ADS"."X_23" AS "X_23", "ADS"."X_24" AS "X_24", "ADS"."X_25" AS "X_25", "ADS"."X_26" AS "X_26", "ADS"."X_27" AS "X_27", "ADS"."X_28" AS "X_28", "ADS"."X_29" AS "X_29", "ADS"."X_30" AS "X_30", "ADS"."X_31" AS "X_31", "ADS"."X_32" AS "X_32", "ADS"."X_33" AS "X_33", "ADS"."X_34" AS "X_34", "ADS"."X_35" AS "X_35", "ADS"."X_36" AS "X_36", "ADS"."X_37" AS "X_37", "ADS"."X_38" AS "X_38", "ADS"."X_39" AS "X_39", "ADS"."X_40" AS "X_40", "ADS"."

# Execute the SQL Code

In [20]:
# save the dataset in a database table


import sqlalchemy as sa

# engine = sa.create_engine('sqlite://' , echo=False)
engine = sa.create_engine("postgresql://db:db@localhost/db?port=5432", echo=False)
conn = engine.connect()
NR = x_test.shape[0]
lTable = pd.DataFrame(x_test.reshape(NR , NC));
lTable.columns = lMetaData['features']
lTable['TGT'] = None
lTable['KEY'] = range(NR)
lTable.to_sql(lMetaData['table'] , conn,   if_exists='replace', index=False)

In [21]:
sql_output = pd.read_sql(lSQL , conn);
sql_output = sql_output.sort_values(by='KEY').reset_index(drop=True)
conn.close()

In [22]:
sql_output.sample(12, random_state=1960)

,KEY,Score_0,Score_1,Score_2,Score_3,Score_4,Score_5,Score_6,Score_7,Score_8,...,LogProba_2,LogProba_3,LogProba_4,LogProba_5,LogProba_6,LogProba_7,LogProba_8,LogProba_9,Decision,DecisionProba
182,182,None,None,None,None,None,None,None,None,None,...,-4.774243,-5.791900,-7.644456,-4.077591,-2.565940,-7.910864,-3.392676,-7.037879,0,0.859310
417,417,None,None,None,None,None,None,None,None,None,...,-13.439655,-11.974681,-17.331422,-8.668499,-9.683689,-15.411725,-12.624130,-15.893189,0,0.999754
820,820,None,None,None,None,None,None,None,None,None,...,-0.001113,-8.986745,-9.725425,-11.544037,-7.055754,-14.077974,-10.435244,-13.522032,2,0.998888
127,127,None,None,None,None,None,None,None,None,None,...,-5.379741,-6.546800,-8.056947,-6.507073,-8.968831,-9.767593,-0.008601,-8.888632,8,0.991436
569,569,None,None,None,None,None,None,None,None,None,...,-11.240164,-11.140171,-19.657740,-10.566568,-11.560256,-14.596848,-12.555394,-15.353217,0,0.999933
917,917,None,None,None,None,None,None,None,None,None,...,-0.001072,-7.452666,-13.704780,-11.846695,-10.292700,-11.354737,-7.956277,-13.856695,2,0.998929
401,401,None,None,None,None,None,None,None,None,None,...,-13.238285,-12.930336,-0.001097,-10.398911,-11.435598,-11.719602,-9.052877,-6.986680,4,0.998904
227,227,None,None,None,None,None,None,None,None,None,...,-7.827059,-7.942908,-11.741683,-0.936883,-8.854241,-13.873971,-0.500842,-9.497823,8,0.606020
726,726,None,None,None,None,None,None,None,None,None,...,-8.554024,-7.118721,-5.290669,-6.795740,-11.000610,-0.226900,-6.604506,-1.640172,7,0.797000
382,382,None,None,None,None,None,None,None,None,None,...,-12.666457,-14.641161,-0.000177,-12.776076,-11.982455,-11.203950,-11.613085,-8.869022,4,0.999823


# Keras Prediction

In [23]:
keras_output = pd.DataFrame()
keras_output_key = pd.DataFrame(list(range(x_test.shape[0])), columns=['KEY']);
keras_output_score = pd.DataFrame(columns=['Score_' + str(x) for x in range(num_classes)]);
keras_output_proba = pd.DataFrame(clf.predict_proba(x_test), columns=['Proba_' + str(x) for x in range(num_classes)])
keras_output = pd.concat([keras_output_key, keras_output_score, keras_output_proba] , axis=1)
for class_label in range(num_classes):
    keras_output['LogProba_' + str(class_label)] = np.log(keras_output_proba['Proba_' + str(class_label)])
keras_output['Decision'] = clf.predict(x_test)
keras_output.sample(12, random_state=1960)



1000/1000 [==============================] - 0s 266us/step


,KEY,Score_0,Score_1,Score_2,Score_3,Score_4,Score_5,Score_6,Score_7,Score_8,...,LogProba_1,LogProba_2,LogProba_3,LogProba_4,LogProba_5,LogProba_6,LogProba_7,LogProba_8,LogProba_9,Decision
182,182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-9.792033,-4.774241,-5.791899,-7.644455,-4.077590,-2.565939,-7.910861,-3.392675,-7.037878,0
417,417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-18.759214,-13.439653,-11.974681,-17.331421,-8.668499,-9.683689,-15.411724,-12.624129,-15.893188,0
820,820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-10.738040,-0.001113,-8.986745,-9.725424,-11.544038,-7.055753,-14.077973,-10.435246,-13.522031,2
127,127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-7.897518,-5.379742,-6.546800,-8.056949,-6.507073,-8.968831,-9.767594,-0.008601,-8.888633,8
569,569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-17.644632,-11.240164,-11.140172,-19.657740,-10.566568,-11.560258,-14.596848,-12.555394,-15.353218,0
917,917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-9.967074,-0.001072,-7.452667,-13.704782,-11.846696,-10.292701,-11.354738,-7.956278,-13.856695,2
401,401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-13.782246,-13.238287,-12.930339,-0.001097,-10.398913,-11.435601,-11.719604,-9.052880,-6.986683,4
227,227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-10.047514,-7.827059,-7.942908,-11.741683,-0.936884,-8.854240,-13.873972,-0.500841,-9.497823,8
726,726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-9.621149,-8.554022,-7.118720,-5.290668,-6.795738,-11.000608,-0.226901,-6.604505,-1.640172,7
382,382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-14.634289,-12.666454,-14.641159,-0.000177,-12.776073,-11.982453,-11.203949,-11.613083,-8.869020,4


# Comparing the SQL and Keras Predictions

In [24]:
sql_keras_join = keras_output.join(sql_output , how='left', on='KEY', lsuffix='_keras', rsuffix='_sql')


In [25]:
sql_keras_join.head(12)

,KEY_keras,Score_0_keras,Score_1_keras,Score_2_keras,Score_3_keras,Score_4_keras,Score_5_keras,Score_6_keras,Score_7_keras,Score_8_keras,...,LogProba_2_sql,LogProba_3_sql,LogProba_4_sql,LogProba_5_sql,LogProba_6_sql,LogProba_7_sql,LogProba_8_sql,LogProba_9_sql,Decision_sql,DecisionProba
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.004121,-8.102524,-15.129297,-11.247557,-10.362124,-9.834754,-8.652616,-13.817639,2,0.995888
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.407755,-0.367492,-8.870429,-4.514480,-5.121749,-10.743680,-3.590343,-10.899488,3,0.692469
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.731996,-6.248429,-7.720157,-4.849324,-4.303448,-8.888398,-0.058248,-7.549072,8,0.943416
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-12.272701,-6.134926,-9.502109,-0.005674,-11.193563,-10.906731,-6.038999,-7.007712,5,0.994342
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-5.125908,-5.569575,-8.289887,-4.570013,-0.029659,-10.968539,-5.867912,-8.995128,6,0.970777
5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-7.006452,-0.762428,-6.318193,-2.201350,-8.722496,-3.603824,-1.438497,-1.901707,3,0.466532
6,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-5.477513,-6.676977,-16.865118,-7.151846,-10.803661,-10.925992,-6.425559,-12.589519,0,0.992115
7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-6.754652,-6.006551,-7.834451,-6.622703,-7.275745,-8.586661,-4.863957,-8.282741,1,0.985785
8,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-6.486332,-4.893933,-9.131959,-0.079503,-7.338284,-4.800388,-3.679651,-7.710565,5,0.923575
9,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-7.410451,-4.923884,-11.705275,-10.850909,-15.140759,-0.015348,-8.883464,-4.938535,7,0.984769


In [26]:
condition = (sql_keras_join.Decision_sql != sql_keras_join.Decision_keras)
sql_keras_join[condition]

,KEY_keras,Score_0_keras,Score_1_keras,Score_2_keras,Score_3_keras,Score_4_keras,Score_5_keras,Score_6_keras,Score_7_keras,Score_8_keras,...,LogProba_2_sql,LogProba_3_sql,LogProba_4_sql,LogProba_5_sql,LogProba_6_sql,LogProba_7_sql,LogProba_8_sql,LogProba_9_sql,Decision_sql,DecisionProba
